## Gradient Descent (경사하강법) 을 구현해보자



필요한 라이브러리 import

In [17]:
import numpy as np 
import sympy as sym  # Symbolic Mathematics
from sympy.abc import x, y # x, y를 변수로 선언 

극값에서 미분값이 0이 되므로 그때까지 이동하면서 극소값을 찾는것이 맞음.

하지만, 컴퓨터로 계산할 때 미분이 정확히 0이 되는 것은 불가능 하므로, eps보다 작을 때 종료하는 조건이 필요하다.

우리는 다음 함수에서 경사하강법을 구현할 것임

`f(x) = x**2 + 2*x + 3`

In [ ]:
def func(val):
    fun = sym.poly(x**2 + 2*x + 3) # sym.poly(): SymPy 다항식 객체 생성
    # fun.subs(x, val): x에 특정 값(val)을 대입한 결과 반환
    return fun.subs(x, val), fun 

def func_gradient(fun, val):
    _, function = fun(val)
    diff = sym.diff(function, x) # sym.diff(function, x) 는 함수의 x에 대한 미분을 계산함
    return diff.subs(x, val), diff

def gradient_descent(fun, init_point, learning_rate=1e-2, epsilon=1e-5):
    cnt = 0
    val = init_point
    diff, _ = func_gradient(fun, init_point)
    while np.abs(diff) > epsilon:
        val = val - learning_rate * diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
        
    print(f"함수 : {fun(val)[1]}, 연산횟수 : {cnt}, 최소점 : ({val}, {fun(val)[0]})")

In [ ]:
gradient_descent(fun = func, init_point = np.random.uniform(-2,2)) #init point : 시작점

함수 : Poly(x**2 + 2*x + 3, x, domain='ZZ'), 연산횟수 : 568, 최소점 : (-1.00000491836263, 2.00000000002419)


---

이번에는 벡터가 입력인 다변수 함수에 대해 편미분하고, 

각 변수별로 편미분을 계산한 gradient 벡터를 이용하여 gradient descent를 구할것임.


`f(x,y) = x**2 + 2*y**2`

벡터는 절대값 대신 norm(노름)을 계산해서 종료조건(eps)을 설정

In [ ]:
def eval_(fun, val):
    val_x, val_y = val
    fun_eval = fun.subs(x, val_x).subs(y, val_y)
    return fun_eval

def func_multi(val):
    x_, y_ = val # symbolic과 겹치지 않기 위해 _ 추가
    func = sym.poly(x**2 + 2*y**2)
    return eval_(func, (x_, y_)), func

def func_gradient(fun, val):
    x_, y_ = val
    _, function = fun(val)
    diff_x = sym.diff(function, x) # x에 대한 편미분
    diff_y = sym.diff(function, y) # y에 대한 편미분
    grad_vec = np.array([eval_(diff_x, (x_, y_)), eval_(diff_y, (x_, y_))], dtype=np.float32)
    return grad_vec, (diff_x, diff_y)

def gradient_descent(fun, init_point, learning_rate=1e-2, epsilon=1e-5):
    cnt=0
    val = init_point
    diff, _ = func_gradient(fun, init_point)
    
    # np.linalg.norm(diff) or np.linalg.norm(diff, ord=2) : L2 norm (유클리드 노름)
    # np.linalg.norm(diff, ord=1) : L1 norm (맨해튼 노름)
    
    while np.linalg.norm(diff) > epsilon:
        val = val - learning_rate * diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
    
    print(f"함수 : {fun(val)[1]}, 연산횟수 : {cnt}, 최소점 : ({val}, {fun(val)[0]})")
    

In [19]:
pt = [np.random.uniform(-2,2), np.random.uniform(-2,2)]
gradient_descent(fun = func_multi, init_point = pt)

함수 : Poly(x**2 + 2*y**2, x, y, domain='ZZ'), 연산횟수 : 554, 최소점 : ([-4.91747969e-06 -1.82817535e-10], 2.41816065913675E-11)
